## Lab 1 - Basic Numeric Tools

* give a situation that requires finding root of a function.  apply best root finder and solve.
* give a pathological function, look for root.
* code trapezoid rule
* Do 2-2 (first couple of functions?).  First find soln as function of # slices.  Then find "accuracy", using the value found above as "correct" value.  Plot as in Ch. 2 Integration notebook.  Determine how accuracy scales with dx (as quoted in book).
* Whiteboards: show how to use Taylor explansion to find eqn for 3-point and 5-point formulae for derivative.
* write a derivative function that acts on data: calc first and second derivative (e.g. to use * to find vel and accel from position.) (e.g. 2-7&2-8)
* examine difference between 3-pt and 5-pt routines on various functions/data?


In [ ]:
%pylab

### Root Finding

Here are a couple of situations where you can use root finding to find an answer.  Pick an appropriate root finder for each case. In a comment, explain briefly why you chose the one you did. 

#### Mortgage Refinance

People usually borrow money from the bank to buy a house, which is called a mortgage.  When you have a mortgage you pay a fixed amount each month.  Your monthly payment was determined at the beginning of the loan, based on the borrowed amount, the length of time for the loan (eg. the number of payments), and the interest rate.  When interest rates drop during the life of a mortgage, you can "re-finance", which means you essentially borrow money to pay off your existing mortgage.  You then make monthly payments on the new loan. The advantage is that the new payments are lower because the interest rate is lower.  The disadvantage is that you are often charged a flat fee ("closing costs") to make the new loan.  So, the accumulated cost of your current mortgage is your current monthly payment times the number of months:

Cost(N) = payment_old * N

while the accumulated cost of the new mortgage would be your new monthly payment times the number of months *plus* the fixed closing cost:

Cost_new(N) = payment_new * N + closing.

Initially, Cost_new will be larger than Cost_old, because of the closing cost.  But over time, the accumulated cost of the new mortgage will be less than teh old mortgage, because of the lower monthly payments.  (Plot each of these functions to convince yourself this is true.)  For the following parameters, find your "break even time", the number of payments at which the new mortgage starts to be less than the old mortgage: 
* Old monthly payment = $ 2,200

* New monthly payment = $ 1,600

* Closing costs = $ 10,000


#### Vertical drop with air resistance

The vertical distance dropped in the presence of air resistance is given by:

eqn

find the time to drop 20 meters.